In [1]:
import autogen

config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": "OPENAI_KEY"
    }
]

assistant_config = {
   "config_list": config_list,
   "temperature": 0,
}

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [4]:
company = {
   "name": "Media Inc",
   "description": "Media Inc., doing business as PlayOn! Sports, is a United States-based high school sports media company. \
                    The Company is composed of six operating divisions: the NFHS Network, PlayOn Sports Properties, PlayOn Sports Technology, \
                    PlayOn Sports Productions, School Broadcast Program and Digital Scout. The Company, through NFHS Network, \
                    streams high school games and live events; PlayOn Sports Properties, partners with state high school associations \
                    to package media rights with marketing assets, and PlayOn Sports Technology, operates a technology platform that \
                    enables productions of live sports events.",
   "location": "Atlanta, Georgia",
   "industry": "Information Technology",
   "stage": "Early growth stage",
   "valuation": "Unknown"
}

lead = {
   "name": "Hamilton Risk Management Co.",
   "description": "Hamilton Risk Management Co. specializes in identifying high-growth companies in the media, technology, and financial sectors. \
                    With expertise in risk analysis and market forecasting, the firm supports its portfolio companies by providing strategic insights and access to growth capital.",
   "location": "New York, New York",
   "aum": "10 billion USD",
   "focus": "Media, technology, and financial sectors",
   "stage_preference": "Early to mid-growth stages",
   "geo_focus": "North America and Europe"
}

candidates = [
   {
       "name": "Buckhead Strategic Corp II",
       "description": "Buckhead Strategic Corp II is a growth-stage investment firm specializing in digital innovation and disruptive media platforms.",
       "location": "Atlanta, Georgia",
       "aum": "800 million USD",
       "focus": "Digital media and innovation",
       "stage": "Early growth stage",
       "geo": "Southeastern United States"
   },
   {
       "name": "Imlay Investments Inc.",
       "description": "Imlay Investments Inc. focuses on high-tech startups in media and broadcasting, offering both financial and operational support.",
       "location": "Atlanta, Georgia",
       "aum": "1 billion USD",
       "focus": "Technology and media startups",
       "stage": "Mid-growth stage",
       "geo": "North America"
   },
   {
       "name": "NMP Management Corp",
       "description": "NMP Management Corp supports scalable businesses in digital broadcasting and media, with an emphasis on emerging technologies.",
       "location": "Atlanta, Georgia",
       "aum": "2 billion USD",
       "focus": "Scalable digital media",
       "stage": "Late growth stage",
       "geo": "North America"
   },
   {
       "name": "Felicis Ventures LLC",
       "description": "Felicis Ventures LLC is a global investment firm focusing on fostering innovation in media, technology, and data-driven platforms.",
       "location": "Menlo Park, California",
       "aum": "3.5 billion USD",
       "focus": "Media and data-driven platforms",
       "stage": "Mid-growth stage",
       "geo": "Global"
   },
   {
       "name": "New Enterprise Associates Inc",
       "description": "New Enterprise Associates Inc invests in transformative technologies across various industries, including media and communications.",
       "location": "Baltimore, Maryland",
       "aum": "15 billion USD",
       "focus": "Transformative technologies and communications",
       "stage": "Early to mid-growth stage",
       "geo": "Global"
   }
]

In [5]:
def create_lead_agent(lead_firm_details, company_details):
    system_message = f"""You are {lead_firm_details['name']}, leading the investment partnership selection.
    Your details:
    - Description: {lead_firm_details['description']}
    - Location: {lead_firm_details['location']}
    - AUM: {lead_firm_details['aum']}
    - Investment Focus: {lead_firm_details['focus']}
    - Stage Preference: {lead_firm_details['stage_preference']}
    - Geographic Focus: {lead_firm_details['geo_focus']}

    The company being evaluated:
    - Name: {company_details['name']}
    - Description: {company_details['description']}
    - Location: {company_details['location']}
    - Industry: {company_details['industry']}
    - Stage: {company_details['stage']}
    - Valuation: {company_details['valuation']}

    Guidelines:
    1. Start each round by following the defined process.
    2. Evaluate proposals based on:
         - Stage Alignment
         - Sector Expertise
         - Geographic Coverage
         - Strategic Value-Add
         - Co-investment History
    3. Finalize partnerships with explicit agreement from the selected candidates.
    4. End the process after three partnerships are finalized or after four rounds.

    Maintain professionalism and focus on the selection process."""
    return autogen.AssistantAgent(
        name="Lead_Firm",
        system_message=system_message,
        llm_config=assistant_config
    )

def create_candidate_agent(candidate_details, company_details, index):
    system_message = f"""You are {candidate_details['name']}, a candidate investment firm.
    Your details:
    - Description: {candidate_details['description']}
    - Location: {candidate_details['location']}
    - AUM: {candidate_details['aum']}
    - Focus: {candidate_details['focus']}
    - Stage: {candidate_details['stage']}
    - Geographic Coverage: {candidate_details['geo']}

    The company being evaluated:
    - Name: {company_details['name']}
    - Description: {company_details['description']}
    - Location: {company_details['location']}
    - Industry: {company_details['industry']}
    - Stage: {company_details['stage']}
    - Valuation: {company_details['valuation']}

    Guidelines:
    1. Present a compelling proposal for partnership in Round 1.
    2. Engage in discussions in Round 2 to improve your chances or decide to quit.
    3. Respond to partnership decisions in Round 3 and 4 as necessary.
    4. Exit the process once selected and finalized.

    Maintain professionalism and focus on the selection process."""
    return autogen.AssistantAgent(
        name=f"Candidate_{index}",
        system_message=system_message,
        llm_config=assistant_config
    )


def process_round(manager, round_number, finalized_partnerships):
    lead_agent = manager.groupchat.agents[0]

    if round_number == 1:
        # Round 1: Candidates present initial proposals
        lead_message = "Candidates, please present your initial proposals for partnership. Ensure your proposals include the format: 'Proposal: [Summary of your proposal]'."
        manager.run_chat(
            messages=[{"role": "assistant", "content": lead_message}],
            sender=lead_agent,
            config=manager.groupchat
        )

    elif round_number == 2:
        # Round 2: Lead firm discusses with candidates
        for candidate_agent in manager.groupchat.agents[1:]:
            discussion_message = f"{candidate_agent.name}, please discuss your proposal further and indicate your willingness to continue."
            manager.run_chat(
                messages=[{"role": "assistant", "content": discussion_message}],
                sender=lead_agent,
                config=manager.groupchat
            )

    elif round_number == 3:
        # Round 3: Lead firm selects 3 candidates and finalizes partnerships
        lead_message = (
            "Lead firm, please provide your selection of up to three candidates for partnership in the format: "
            "'Selection: [Candidate 1], [Candidate 2], [Candidate 3]'."
        )
        manager.run_chat(
            messages=[{"role": "assistant", "content": lead_message}],
            sender=lead_agent,
            config=manager.groupchat
        )

        # Parse the lead firm's selection
        selected_candidates = []
        for msg in manager.groupchat.messages:
            if "selection:" in msg.get("content", "").lower():
                content = msg["content"].lower().replace("selection:", "").strip()
                selected_candidates = [name.strip() for name in content.split(",")]
                break

        agreements = finalize_partnership(manager, selected_candidates, finalized_partnerships)

        return "Process concluded: Three partnerships finalized." if len(finalized_partnerships) == 3 else finalized_partnerships

    return finalized_partnerships


def finalize_partnership(manager, selected_candidates, finalized_partnerships):
    agreements = []
    for candidate in selected_candidates:
        candidate_agent = next(
            (a for a in manager.groupchat.agents if a.name == candidate), None
        )
        if not candidate_agent:
            continue

        # Send the agreement message to the candidate
        manager.run_chat(
            messages=[{"role": "assistant", "content": f"{candidate}, do you agree to the partnership?"}],
            sender=candidate_agent,
            config=manager.groupchat
        )

        # Check candidate's response
        for msg in manager.groupchat.messages:
            if msg['sender'] == candidate_agent.name and "agree" in msg['content'].lower():
                agreements.append(candidate)
                finalized_partnerships.append(candidate)
                manager.groupchat.agents = [
                    a for a in manager.groupchat.agents if a.name != candidate
                ]
                break

    return agreements


def run_partnership_selection(company_details, lead_firm_details, candidate_details_list):
    lead_agent = create_lead_agent(lead_firm_details, company_details)
    candidate_agents = [create_candidate_agent(c, company_details, i) for i, c in enumerate(candidate_details_list, 1)]
    finalized_partnerships = []

    groupchat = autogen.GroupChat(
        agents=[lead_agent] + candidate_agents,
        max_round=30,
        messages=[],
        speaker_selection_method="round_robin"
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=assistant_config)

    manager.initiate_chat(
        lead_agent, message=f"Starting partnership selection process for the company: {company_details['name']}"
    )

    for round_number in range(1, 4):
        result = process_round(manager, round_number, finalized_partnerships)
        if "Process concluded" in result:
            break

    return finalized_partnerships

In [6]:
run_partnership_selection(company, lead, candidates)

chat_manager (to Lead_Firm):

Starting partnership selection process for the company: Media Inc

--------------------------------------------------------------------------------
Lead_Firm (to chat_manager):

Thank you for initiating the partnership selection process for Media Inc. We will evaluate the proposal based on the defined criteria. 

### Evaluation Criteria:
1. **Stage Alignment**: Media Inc is in the early growth stage, which aligns with our preference for early to mid-growth stages.
2. **Sector Expertise**: Media Inc operates in the information technology sector, specifically focusing on high school sports media. While this is a niche within the broader media and technology sectors, it does align with our investment focus.
3. **Geographic Coverage**: Media Inc is based in Atlanta, Georgia, which falls within our geographic focus of North America.
4. **Strategic Value-Add**: Media Inc's operations, including streaming high school games and partnering with state high school as

NameError: name 'cerebras_AuthenticationError' is not defined